# Lesson 1: Router Engine

Welcome to Lesson 1.

To access the `requirements.txt` file, the data/pdf file required for this lesson and the `helper` and `utils` modules, please go to the `File` menu and select`Open...`.

I hope you enjoy this course!

## Setup

In [1]:
from helper import get_openai_api_key, get_api_version, get_openai_endpoint

AZURE_OPENAI_API_KEY = get_openai_api_key()
OPENAI_API_VERSION = get_api_version()
AZURE_OPENAI_ENDPOINT = get_openai_endpoint()

In [2]:
print(AZURE_OPENAI_API_KEY)

In [3]:
print(OPENAI_API_VERSION)

2024-02-15-preview


In [4]:
import nest_asyncio

nest_asyncio.apply()

## Load Data

To download this paper, below is the needed code:

#!wget "https://openreview.net/pdf?id=VtmBAGCN7o" -O metagpt.pdf

**Note**: The pdf file is included with this lesson. To access it, go to the `File` menu and select`Open...`.

In [5]:
# !pip install -r requirements.txt

In [6]:
from llama_index.core import SimpleDirectoryReader

# load documents
documents = SimpleDirectoryReader(input_files=["metagpt.pdf"]).load_data()

## Define LLM and Embedding model

In [7]:
from llama_index.core.node_parser import SentenceSplitter

splitter = SentenceSplitter(chunk_size=1024)
nodes = splitter.get_nodes_from_documents(documents)

In [8]:
from llama_index.core import Settings
from llama_index.llms.azure_openai import AzureOpenAI
from llama_index.embeddings.azure_openai import AzureOpenAIEmbedding


# Assuming 'azure-gpt-3.5-turbo' and 'azure-text-embedding-ada-002' are valid model identifiers in AzureOpenAI
Settings.llm = AzureOpenAI(
    engine="gpt-35-turbo",
    azure_endpoint=AZURE_OPENAI_ENDPOINT,
    api_version=OPENAI_API_VERSION
    )

Settings.embed_model = AzureOpenAIEmbedding(
    engine="text-embedding-ada-002",
    azure_endpoint=AZURE_OPENAI_ENDPOINT,
    api_version=OPENAI_API_VERSION   
    )


In [10]:
import pprint


pprint.pprint(Settings.llm)

AzureOpenAI(callback_manager=<llama_index.core.callbacks.base.CallbackManager object at 0x308f39e50>, system_prompt=None, messages_to_prompt=<function messages_to_prompt at 0x147c8a0c0>, completion_to_prompt=<function default_completion_to_prompt at 0x147d282c0>, output_parser=None, pydantic_program_mode=<PydanticProgramMode.DEFAULT: 'default'>, query_wrapper_prompt=None, model='gpt-35-turbo', temperature=0.1, max_tokens=None, logprobs=None, top_logprobs=0, additional_kwargs={}, max_retries=3, timeout=60.0, default_headers=None, reuse_client=True, api_key='', api_base='https://api.openai.com/v1', api_version='2024-02-15-preview', engine='gpt-35-turbo', azure_endpoint='https://my-openai-ce.openai.azure.com/', azure_deployment=None, use_azure_ad=False, azure_ad_token_provider=None)


## Define Summary Index and Vector Index over the Same Data

In [11]:
from llama_index.core import SummaryIndex, VectorStoreIndex

summary_index = SummaryIndex(nodes)


In [12]:
vector_index = VectorStoreIndex(nodes)

## Define Query Engines and Set Metadata

In [13]:
summary_query_engine = summary_index.as_query_engine(
    response_mode="tree_summarize",
    use_async=True,
)
vector_query_engine = vector_index.as_query_engine()

In [14]:
from llama_index.core.tools import QueryEngineTool


summary_tool = QueryEngineTool.from_defaults(
    query_engine=summary_query_engine,
    description=(
        "Useful for summarization questions related to MetaGPT"
    ),
)

vector_tool = QueryEngineTool.from_defaults(
    query_engine=vector_query_engine,
    description=(
        "Useful for retrieving specific context from the MetaGPT paper."
    ),
)

## Define Router Query Engine

In [15]:
from llama_index.core.query_engine.router_query_engine import RouterQueryEngine
from llama_index.core.selectors import LLMSingleSelector


query_engine = RouterQueryEngine(
    selector=LLMSingleSelector.from_defaults(),
    query_engine_tools=[
        summary_tool,
        vector_tool,
    ],
    verbose=True
)

In [16]:
response = query_engine.query("What is the summary of the document?")
print(str(response))

Selecting query engine 0: This choice indicates that the document is useful for summarization questions related to MetaGPT, which aligns with the question asking for the summary of the document..


RateLimitError: Error code: 429 - {'error': {'code': '429', 'message': 'Requests to the ChatCompletions_Create Operation under Azure OpenAI API version 2024-02-15-preview have exceeded call rate limit of your current OpenAI S0 pricing tier. Please retry after 10 seconds. Please go here: https://aka.ms/oai/quotaincrease if you would like to further increase the default rate limit.'}}

In [ ]:
print(len(response.source_nodes))

In [ ]:
response = query_engine.query(
    "How do agents share information with other agents?"
)
print(str(response))

## Let's put everything together

In [ ]:
from utils import get_router_query_engine

query_engine = get_router_query_engine("metagpt.pdf")

In [ ]:
response = query_engine.query("Tell me about the ablation study results?")
print(str(response))